In [1]:
import os
import cv2
import numpy as np
import random
import torch
import torch.optim as optim
import torchvision.datasets
import torchvision.transforms as T
from torchinfo import summary
# from torchsummary import summary 

In [2]:
MNIST_train = torchvision.datasets.MNIST(root='MNIST_data/', train=True, transform=T.ToTensor(), download=True)

In [3]:
MNIST_test = torchvision.datasets.MNIST(root='MNIST_data/', train=False, transform=T.ToTensor(), download=True)

In [4]:
MNIST_train.data.size()

torch.Size([60000, 28, 28])

In [5]:
MNIST_test.data.size()

torch.Size([10000, 28, 28])

In [ ]:
MNIST_train.data[0]

In [6]:
print(MNIST_train.targets[0])
sample_img = MNIST_train.data[0].numpy()
sample_img = cv2.resize(sample_img, (256, 256))
cv2.imshow('MNIST', sample_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

tensor(5)


img, label = next(iter(MNIST_train))

In [7]:
img, label = next(iter(MNIST_train))

In [8]:
img

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [9]:
label

5

In [10]:
batch_size = 100
training_epochs = 15
learning_rate = 0.001

In [11]:
data_loader = torch.utils.data.DataLoader(MNIST_train, batch_size=batch_size, shuffle=True)

In [12]:
data = next(iter(data_loader))
data[0] #  이미지배차서아주 먼쿰 거져옴
data[0][1] # 가져와서 0번째
data[1] # 레이블을 배치사이즈만큼 
data[1][0] # 가져온거 0번째

tensor(7)

In [13]:
total_batch = len(data_loader)
total_batch

600

In [14]:
a = torch.arange(10)
a.view(2,5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [15]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=128, out_channels=625, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc_layer = torch.nn.Linear(2 * 2 * 625, 10, bias=True)

    def forward(self, x):
        L1 = self.layer1(x)
        L2 = self.layer2(L1)
        L3 = self.layer3(L2)
        L4 = self.layer4(L3)
        fc_input = L4.view(L4.size(0), -1)
        fc = self.fc_layer(fc_input)
        return fc

In [16]:
model = CNN()

In [17]:
w1 = next(model.layer1.parameters())
w1.size()

torch.Size([32, 1, 3, 3])

summary(model, (1, 1, 28, 28))

In [18]:
summary(model, (1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      --                        --
├─Sequential: 1-1                        [1, 32, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─ReLU: 2-2                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-3                    [1, 32, 14, 14]           --
├─Sequential: 1-2                        [1, 64, 7, 7]             --
│    └─Conv2d: 2-4                       [1, 64, 14, 14]           18,496
│    └─ReLU: 2-5                         [1, 64, 14, 14]           --
│    └─MaxPool2d: 2-6                    [1, 64, 7, 7]             --
├─Sequential: 1-3                        [1, 128, 3, 3]            --
│    └─Conv2d: 2-7                       [1, 128, 7, 7]            73,856
│    └─ReLU: 2-8                         [1, 128, 7, 7]            --
│    └─MaxPool2d: 2-9                    [1, 128, 3, 3]            --
├─Sequ

In [19]:
loss_func = torch.nn.CrossEntropyLoss()

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)